Import the neccessary libraries

In [ ]:
import ternary
from ternary.helpers import simplex_iterator
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolDescriptors
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm

Specify the inputs

In [ ]:
#csv file should have at least two populated columns titled "ID" and "Smiles"
input_filepath = 'Notebook Outputs/Combined_DELs.csv' 


Create the ternary list of Normalized PMI Ratios

In [ ]:

#Process the input csv file
df=pd.read_csv(input_filepath) 
DEL_list=df.DEL.unique() #Create a list of members in the "DEL" column
libraries=df['DEL']
IDs=df['ID']
Smiles=df['Smiles']
library_list=libraries.values.tolist()
ID_list=IDs.values.tolist()
smiles_list=Smiles.values.tolist()

#Initialize Error Counters
optimization_errors=0
conformer_errors=0

#For each compound, perform a 3D UFF optimization and then calculate the normalized PMI ratios
print('Queued ' + str(len(Smiles))+ ' compounds for optimization.')
print('This will take a few minutes.')
print('Performing 3D UFF optimizations...')
table=pd.DataFrame()
for i,smiles in tqdm(enumerate(smiles_list)):
    library=library_list[i]
    id=ID_list[i]
    mol=Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    Chem.SanitizeMol(mol)
    status = AllChem.EmbedMolecule(mol)
    try:
        status = AllChem.UFFOptimizeMolecule(mol)
    except ValueError:
        optimization_errors+=1
        continue
    try:
        conformer = mol.GetConformer()
    except ValueError:
        conformer_errors+=1
        continue
    table.loc[i,'DEL']=library
    table.loc[i,'ID']=id
    table.loc[i,'NPR1']=rdMolDescriptors.CalcNPR1(mol)
    table.loc[i,'NPR2']=rdMolDescriptors.CalcNPR2(mol)

ternary_list=[]

#Use the NPR values to calculate values for sphere-, disk-, and rod-likeness
for npr1, npr2 in zip(table['NPR1'], table['NPR2']):
    if np.isnan(npr1): continue #skip values that are not numbers
    npr1=round(npr1, 2)
    npr2=round(npr2, 2)
    sphere_likeness = (npr1+npr2)-1
    rod_likeness =  (npr2-npr1)
    disc_likeness = 2-(2*npr2)
    ternary_list.append([sphere_likeness, disc_likeness, rod_likeness])

#Organize the data into a dataframe for optional output
for i, [sphere_likeness, disc_likeness, rod_likeness] in enumerate(ternary_list):
    table.loc[i,'Sphere']=sphere_likeness
    table.loc[i,'Rod']=rod_likeness
    table.loc[i,'Disk']=disc_likeness
input_len=len(smiles_list)
output_len=len(ternary_list)
print (( str(input_len)) + " / " + str(output_len) + " successful completions")
print ('Skipped optimization errors: ' + str(optimization_errors))
print ('Skipped conformer errors: ' + str(conformer_errors))

Create a csv output of the NPR values (optional)

In [ ]:
table.to_csv('Notebook Outputs/PMI_Test.csv')

Generate the Plots

In [ ]:

#Add the ternary data to a dataframe grouped by DEL
ternary_df=pd.DataFrame()
for type in DEL_list:
        temp_ternary_list=[]
        temp_df=table.loc[table['DEL'] == type]
        sphere=temp_df['Sphere'].values.tolist()
        disc=temp_df['Disk'].values.tolist()
        rod=temp_df['Rod'].values.tolist()
        temp_ternary_list=list(zip(sphere, disc, rod))

        output_filepath="Notebook Outputs/"+str(type) + "Plot.eps"
        title=str(type) + " PMI Plot"

        # Split data into bins with width 1/scale
        scale = 25 # number of bins along each axis
        bins = np.linspace(0,1,scale+1)[1:]

        # digitize returns the index of the bin to which a value was assigned
        x_inds = list(np.digitize(np.array([i[0] for i in temp_ternary_list]), bins))
        y_inds = list(np.digitize(np.array([i[1] for i in temp_ternary_list]), bins))
        z_inds = list(np.digitize(np.array([i[2] for i in temp_ternary_list]), bins))
        ternary_list_binned = list(zip(x_inds, y_inds, z_inds))

        # Populate ternary_dict with {(i,j,k):frequency}
        ternary_dict = dict()

        # Initiate all possible (i,j,k) vertices (only keep i and j as k is implied by these)
        for (i,j,k) in simplex_iterator(scale):
                ternary_dict[(i,j)] = 0

        # Count number of occurences of each (i,j,k) in binned data
        for i,j,k in ternary_list_binned:
                ternary_dict[(i,j)]+=1

        figure, ax = plt.subplots()

        tax = ternary.TernaryAxesSubplot(ax=ax, scale=scale)

        offset=0

        # Sizing, borders and title
        plt.axis('off') #turn border off
        figure.set_size_inches(5, 4)
        tax.set_title(title, fontsize=12)
        tax.boundary(linewidth=1.5)

        # Get colormap with white below vmin (eps)
        cmap_whiteZero =matplotlib.colormaps['Spectral_r']
        #cmap_whiteZero = matplotlib.cm.get_cmap('Spectral_r')
        cmap_whiteZero.set_under('w')
        eps = np.spacing(0.0)

        # Plot ternary heatmap
        tax.heatmap(ternary_dict, style="t", cmap=cmap_whiteZero, vmin=eps, colorbar=True) # style can be t, d, or h (see documentation)
        figure.gca().invert_yaxis() #turn plot upside down

        # Save figure 
        plt.savefig(output_filepath)